In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.swmhau_network_functions import (
    swmhau_network_hyperparameter_search,
)

In [4]:
output_dir = "rumours_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## Rumours

In [5]:
%run load_sbert-embeddings.py

In [6]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


# swmhau Network

In [7]:
features = ["time_encoding", "timeline_index"]
standardise_method = ["z_score", None]
num_features = len(features)
add_time_in_path = True

In [8]:
num_epochs = 100
embedding_dim = 384
dimensions = [15]  # [50, 15]
# define swmhau parameters: (output_channels, sig_depth, num_heads)
swmhau_parameters = [(12, 3, 10), (8, 4, 6), (8, 4, 12)]
num_layers = [1]
ffn_hidden_dim_sizes = [[256, 256], [512, 512]]
dropout_rates = [0.5, 0.1]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5
split_indices = (
    df_rumours[df_rumours["set"] == "train"].index,
    df_rumours[df_rumours["set"] == "dev"].index,
    df_rumours[df_rumours["set"] == "test"].index,
)

## UMAP

In [12]:
size = 35
(
    swmhau_network_umap,
    best_swmhau_network_umap,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    embedding_dim=embedding_dim,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swmhau_parameters=swmhau_parameters,
    num_layers=num_layers,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    split_indices=split_indices,
    k_fold=False,
    features=features,
    standardise_method=standardise_method,
    add_time_in_path=add_time_in_path,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}.csv",
    verbose=False,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swmhau_network_umap_focal_2_35.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swmhau_network_umap_focal_2_35_best_model.csv


In [13]:
swmhau_network_umap.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_3959843/3742016164.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_umap.groupby(["dimensions",


loss  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (256, 256)     0.0001         0.371428   
                                                                         0.0005         0.316856   
                                                                         0.0010         0.384461   
                                                          (512, 512)     0.0001         0.432205   
                                                                         0.0005         0.365014   
                                                                         0.0010         0.348443   
                                     12        1          (256, 256)     0.0001         0.387116   
                                                                         0.0005         0.342072   
                                                                         0.0010         0.328106   
                                                          (512, 512)     0.0001         0.360836   
                                                                         0.0005         0.337853   
                                                                         0.0010         0.371939   
           12              3         10        1          (256, 256)     0.0001         0.441040   
                                                                         0.0005         0.427338   
                                                                         0.0010         0.383594   
                                                          (512, 512)     0.0001         0.456059   
                                                                         0.0005         0.577284   
                                                                         0.0010         0.389388   

                                                                                        accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (256, 256)     0.0001         0.632666   
                                                                         0.0005         0.622974   
                                                                         0.0010         0.618843   
                                                          (512, 512)     0.0001         0.629171   
                                                                         0.0005         0.625040   
                                                                         0.0010         0.600890   
                                     12        1          (256, 256)     0.0001         0.622815   
                                                                         0.0005         0.643470   
                                                                         0.0010         0.621703   
                                                          (512, 512)     0.0001         0.637274   
                                                                         0.0005         0.620750   
                                                                         0.0010         0.623610   
           12              3         10        1          (256, 256)     0.0001         0.631077   
                                                                         0.0005         0.615983   
                                                                         0.0010         0.612806   
                                                          (512, 512)     0.0001         0.624245   
                                                                         0.0005         0.625993   
                                                                         0.0010         0.622339   

                                                                                          

In [14]:
best_swmhau_network_umap

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,0.370341,0.615825,0.595707,"[0.5055214723926381, 0.6858924395947]",0.668189,"[0.7744360902255639, 0.561941251596424]",0.627614,"[0.37522768670309653, 0.88]",0.220668,0.672598,...,0.0001,1,focal,2,False,None,Conv1d,None,concatenation,64
0,0.446978,0.626311,0.623477,"[0.5908141962421712, 0.656140350877193]",0.638153,"[0.6919315403422983, 0.584375]",0.631741,"[0.5154826958105647, 0.748]",0.300115,0.661922,...,0.0001,12,focal,2,False,None,Conv1d,None,concatenation,64
0,0.589764,0.613918,0.613298,"[0.6287809349220898, 0.5978152929493545]",0.613265,"[0.6328413284132841, 0.5936883629191322]",0.613386,"[0.6247723132969034, 0.602]",0.243468,0.722420,...,0.0001,123,focal,2,False,None,Conv1d,None,concatenation,64


In [15]:
best_swmhau_network_umap[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

,dimensions,output_channels,sig_depth,num_heads,num_layers,ffn_hidden_dim,dropout_rate,learning_rate
0,15,12,3,10,1,"(512, 512)",0.1,0.0001
0,15,12,3,10,1,"(512, 512)",0.1,0.0001
0,15,12,3,10,1,"(512, 512)",0.1,0.0001


In [16]:
best_swmhau_network_umap["f1"].mean()

0.6108274478296911

In [17]:
best_swmhau_network_umap["precision"].mean()

0.6398689289161171

In [18]:
best_swmhau_network_umap["recall"].mean()

0.6242471159684274

In [19]:
np.stack(best_swmhau_network_umap["f1_scores"]).mean(axis=0)

array([0.57503887, 0.64661603])

In [20]:
np.stack(best_swmhau_network_umap["precision_scores"]).mean(axis=0)

array([0.69973632, 0.58000154])

In [21]:
np.stack(best_swmhau_network_umap["recall_scores"]).mean(axis=0)

array([0.5051609 , 0.74333333])

## Random Projections

In [9]:
size = 35
(
    swmhau_network_grp,
    best_swmhau_network_grp,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    embedding_dim=embedding_dim,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swmhau_parameters=swmhau_parameters,
    num_layers=num_layers,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    split_indices=split_indices,
    k_fold=False,
    features=features,
    standardise_method=standardise_method,
    add_time_in_path=add_time_in_path,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}.csv",
    verbose=False,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swmhau_network_grp_focal_2_35.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swmhau_network_grp_focal_2_35_best_model.csv


In [10]:
swmhau_network_grp.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_2974182/3837375781.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_grp.groupby(["dimensions",


loss  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.416057   
                                                                                      0.0005         0.747329   
                                                                                      0.0010         0.478982   
                                                                         0.5          0.0001         0.372697   
                                                                                      0.0005         0.396219   
                                                                                      0.0010         0.454991   
                                                          (512, 512)     0.1          0.0001         0.487213   
                                                                                      0.0005         0.475595   
                                                                                      0.0010         0.478557   
                                                                         0.5          0.0001         0.355497   
                                                                                      0.0005         0.528796   
                                                                                      0.0010         0.486589   
                                     12        1          (256, 256)     0.1          0.0001         0.326538   
                                                                                      0.0005         0.423894   
                                                                                      0.0010         0.382555   
                                                                         0.5          0.0001         0.361878   
                                                                                      0.0005         0.567318   
                                                                                      0.0010         0.421517   
                                                          (512, 512)     0.1          0.0001         0.842868   
                                                                                      0.0005         0.558859   
                                                                                      0.0010         0.492024   
                                                                         0.5          0.0001         0.403736   
                                                                                      0.0005         0.620097   
                                                                                      0.0010         0.401416   
           12              3         10        1          (256, 256)     0.1          0.0001         0.391251   
                                                                                      0.0005         0.398131   
                                                                                      0.0010         0.429065   
                                                                         0.5          0.0001         0.413059   
                                                                                      0.0005         0.393927   
                                                                                      0.0010         0.551666   
                                                          (512, 512)     0.1          0.0001         0.379375   
                                                                                      0.0005         0.374126   
                                                                                      0.0010         0.747928   
                                                                         0.5          0.0001         0.419185   
                                     

In [11]:
best_swmhau_network_grp

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,0.338599,0.626311,0.620928,"[0.5757575757575758, 0.666098807495741]",0.644726,"[0.7093333333333334, 0.5801186943620178]",0.633259,"[0.48451730418943534, 0.782]",0.230089,0.683274,...,0.0001,1,focal,2,False,None,Conv1d,None,concatenation,64
0,0.934016,0.585319,0.574930,"[0.6413849958779885, 0.5084745762711864]",0.585129,"[0.5858433734939759, 0.5844155844155844]",0.579281,"[0.7085610200364298, 0.45]",0.224750,0.814947,...,0.0001,12,focal,2,False,None,Conv1d,None,concatenation,64
0,1.255990,0.616778,0.614592,"[0.6436170212765957, 0.5855670103092784]",0.615599,"[0.6269430051813472, 0.6042553191489362]",0.614601,"[0.6612021857923497, 0.568]",0.494560,0.765125,...,0.0001,123,focal,2,False,None,Conv1d,None,concatenation,64


In [12]:
best_swmhau_network_grp.columns

Index(['loss', 'accuracy', 'f1', 'f1_scores', 'precision', 'precision_scores',
       'recall', 'recall_scores', 'valid_loss', 'valid_accuracy', 'valid_f1',
       'valid_f1_scores', 'valid_precision', 'valid_precision_scores',
       'valid_recall', 'valid_recall_scores', 'k', 'dimensions', 'sig_depth',
       'method', 'input_channels', 'output_channels', 'features',
       'standardise_method', 'add_time_in_path', 'num_features',
       'embedding_dim', 'log_signature', 'num_heads', 'num_layers',
       'ffn_hidden_dim', 'dropout_rate', 'learning_rate', 'seed',
       'loss_function', 'gamma', 'k_fold', 'n_splits', 'augmentation_type',
       'hidden_dim_aug', 'comb_method', 'batch_size'],
      dtype='object')

In [13]:
best_swmhau_network_grp[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

,dimensions,output_channels,sig_depth,num_heads,num_layers,ffn_hidden_dim,dropout_rate,learning_rate
0,15,8,4,12,1,"(512, 512)",0.1,0.0001
0,15,8,4,12,1,"(512, 512)",0.1,0.0001
0,15,8,4,12,1,"(512, 512)",0.1,0.0001


In [14]:
best_swmhau_network_grp["f1"].mean()

0.6034833311647276

In [15]:
best_swmhau_network_grp["precision"].mean()

0.6151515516558658

In [16]:
best_swmhau_network_grp["recall"].mean()

0.6090467516697023

In [17]:
np.stack(best_swmhau_network_grp["f1_scores"]).mean(axis=0)

array([0.6202532 , 0.58671346])

In [18]:
np.stack(best_swmhau_network_grp["precision_scores"]).mean(axis=0)

array([0.64070657, 0.58959653])

In [19]:
np.stack(best_swmhau_network_grp["recall_scores"]).mean(axis=0)

array([0.6180935, 0.6      ])